In [4]:
import json
import os
import sys
from collections import defaultdict
from datetime import datetime

import requests
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

from marketAPI import AlphaVantage

%matplotlib inline

import pickle

In [101]:
today = pd.Timestamp.today()

date_str = today.date().strftime("%Y%m%d")
timestamp = today.timestamp()

In [115]:
active_df = pd.read_html("https://finance.yahoo.com/most-active")[0]

In [113]:
robinhood_popular = pd.read_html("https://robinhood.com/collections/100-most-popular")[0]

In [153]:
day_gainers = pd.read_html("https://finance.yahoo.com/screener/predefined/day_gainers")[0]

In [149]:
day_losers = pd.read_html("https://finance.yahoo.com/screener/predefined/day_losers")[0]

In [ ]:
def shortConvert(cap):
    multiplier = cap[-1]
    number = {'M': 1e6, 'B': 1e9, 'T': 1e12}
    
    return float(cap[:-1]) * number.get(multiplier)

In [147]:
def processList(market_df):
    renameDict = {
        'Price (Intraday)': 'Price',
        '% Change': 'PercentChange',
        'Today': 'PercentChange'
    }
    market_df = market_df.rename(columns=renameDict)
    market_df.PercentChange = market_df.PercentChange.apply(lambda x: float(str(x).strip("+%")))
    market_df.loc[:, 'Market Cap'] = market_df.loc[:, 'Market Cap'].apply(shortConvert)
    
    
    return market_df

In [174]:
test = processList(day_gainers)

In [182]:
10e9

10000000000.0

In [186]:
test[test['Market Cap'] > 2e10]

,Symbol,Name,Price,Change,PercentChange,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
3,MRNA,"Moderna, Inc.",94.8500,13.0400,15.94,103.509M,24.942M,3.521100e+10,NaN,NaN
8,ERIC,Telefonaktiebolaget LM Ericsson (publ),10.8900,1.2700,13.20,30.918M,9.492M,3.648300e+10,NaN,NaN
10,ERIXF,Telefonaktiebolaget LM Ericsson (publ),10.8200,1.1600,12.06,17565,237239,3.611500e+10,NaN,NaN
24,ISRG,"Intuitive Surgical, Inc.",650.7400,46.4900,7.69,1.665M,644141,7.597400e+10,56.10,NaN
25,BGNE,"BeiGene, Ltd.",248.6700,17.0800,7.38,570503,264717,2.205900e+10,NaN,NaN
26,TWLO,Twilio Inc.,238.5700,16.1900,7.28,4.501M,4.277M,3.341200e+10,NaN,NaN
29,HNDAF,"Honda Motor Co., Ltd.",26.2600,1.7000,6.94,80000,30319,4.909300e+10,4.53,NaN
32,AZN,AstraZeneca PLC,61.1000,3.6700,6.39,24.976M,5.744M,1.554540e+11,105.62,NaN
37,RLLCF,Rolls-Royce Holdings plc,0.0055,0.0003,5.77,3.914M,5.833M,1.199040e+11,NaN,NaN
50,RPRX,Royalty Pharma plc,44.1900,1.9900,4.72,1.764M,3.749M,2.682500e+10,NaN,NaN


In [171]:
robinhood_popular.iloc[-1].Symbol

'VKTX'

In [159]:
robinhood_popular[['Name', 'Symbol']].to_records(index=False)

rec.array([('Ford Motor', 'F'), ('GE', 'GE'),
           ('American Airlines', 'AAL'), ('Disney', 'DIS'),
           ('Delta Air Lines', 'DAL'), ('Apple', 'AAPL'),
           ('Microsoft', 'MSFT'), ('Carnival', 'CCL'), ('GoPro', 'GPRO'),
           ('Tesla', 'TSLA'), ('Aurora Cannabis', 'ACB'),
           ('Plug Power', 'PLUG'), ('Amazon', 'AMZN'),
           ('Norwegian Cruise Line', 'NCLH'), ('Bank of America', 'BAC'),
           ('Snap', 'SNAP'), ('Fitbit', 'FIT'), ('Boeing', 'BA'),
           ('United Airlines', 'UAL'), ('Moderna', 'MRNA'),
           ('NIO', 'NIO'), ('Uber', 'UBER'), ('Alibaba', 'BABA'),
           ('Canopy Growth', 'CGC'), ('Facebook', 'FB'),
           ('Royal Caribbean', 'RCL'), ('Twitter', 'TWTR'),
           ('AMD', 'AMD'), ('Cronos Group', 'CRON'), ('Inovio', 'INO'),
           ('Zynga', 'ZNGA'), ('Netflix', 'NFLX'),
           ('Spirit Airlines', 'SAVE'), ('Coca-Cola', 'KO'),
           ('AT&T', 'T'), ('Starbucks', 'SBUX'), ('Aphria', 'APHA'),
           ('

In [189]:
processList(day_gainers).sort_values('PercentChange').iloc[0][['Symbol', 'Name']].tolist()

['OHI', 'Omega Healthcare Investors, Inc.']

In [67]:
# active_dict = active_df.set_index("Symbol").T.to_dict()
# active_order = active_df.Symbol.tolist()
# with open(f"activeTickers_{date_str}.pkl", "wb") as f:
#     pickle.dump(active_dict, f)

In [70]:
symbol = "SPCE"
interval="5min"

av = AlphaVantage()
intraday = av.intradayStock(symbol, interval=interval, size="compact")
daily = av.dailyAdjStock(symbol, size='full')

In [81]:
intradayData = intraday[f'Time Series ({interval})']
intradayDict = defaultdict(dict)
for k, v in intradayData.items():
    date_key = datetime.strptime(k, "%Y-%m-%d %H:%M:%S")
    intradayDict[date_key]['close'] = v['4. close']
    intradayDict[date_key]['volume'] = v['5. volume']

dailyData = daily[f'Time Series (Daily)']
dailyDict = defaultdict(dict)
for k, v in dailyData.items():
    date_key = datetime.strptime(k, "%Y-%m-%d")
    dailyDict[date_key]['close'] = v['5. adjusted close']
    dailyDict[date_key]['volume'] = v['6. volume']

In [103]:

if not os.path.exists(f'data_{date_str}'):
    os.mkdir(f'data_{date_str}')
    


In [110]:

(
    pd.DataFrame(dailyDict).T
    .iloc[:200]
    .reset_index()
    .rename(columns={'index': 'date'})
    .to_csv(f'data_{date_str}/{symbol}_daily_{timestamp}.csv', index=False)
)

In [105]:
(
    pd.DataFrame(intradayDict).T
    .reset_index()
    .rename(columns={'index': 'time'})
    .to_csv(f'data_{date_str}/{symbol}_intraday_{timestamp}.csv', index=False)
)